In [1]:
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from PIL import Image
import numpy as np
import os
from elpv_reader import load_dataset

   
def map_probability_to_label(prob):
    if prob == 0:
        return 0
    elif 0 < prob <= 0.4:
        return 1
    elif 0.4 < prob <= 0.8:
        return 2
    else:
        return 3



images, probs, types= load_dataset()
# print(images)
# print(image_height)

y = np.array([map_probability_to_label(prob) for prob in probs])
X = images.reshape(images.shape[0], -1)
# Split dataset into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75, random_state=42,stratify=y)
svm_classifier = svm.SVC(C=0.5 ,kernel='linear',gamma=0.008,decision_function_shape='ovo')
svm_classifier.fit(X_train, y_train)


y_pred = svm_classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[910  64  28 129]
 [112  67   7  35]
 [ 52   7   2  19]
 [191  53  26 266]]
              precision    recall  f1-score   support

           0       0.72      0.80      0.76      1131
           1       0.35      0.30      0.33       221
           2       0.03      0.03      0.03        80
           3       0.59      0.50      0.54       536

    accuracy                           0.63      1968
   macro avg       0.42      0.41      0.41      1968
weighted avg       0.62      0.63      0.62      1968



In [2]:
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from PIL import Image
import numpy as np
import os
from elpv_reader import load_dataset

   
def map_probability_to_label(prob):
    if prob == 0:
        return 0
    elif 0 < prob <= 0.4:
        return 1
    elif 0.4 < prob <= 0.8:
        return 2
    else:
        return 3


images, probs, types = load_dataset()  

# Convert probability to category label
y = np.array([map_probability_to_label(prob) for prob in probs])

# Split the data set by type
mono_indices = types == 'mono'
poly_indices = types == 'poly'

# mono
images_mono = images[mono_indices]
y_mono = y[mono_indices]
images_mono_f = images_mono.reshape(images_mono.shape[0], -1)
images_mono_val = images_mono_f[-100:]
y_mono_val = y_mono[-100:]
images_mono_train_test = images_mono_f[:-100]
y_mono_train_test = y_mono[:-100]
# poly
images_poly = images[poly_indices]
y_poly = y[poly_indices]
images_poly_f = images_poly.reshape(images_poly.shape[0], -1)
images_poly_val = images_poly_f[-100:]
y_poly_val = y_poly[-100:]
images_poly_train_test = images_poly_f[:-100]
y_poly_train_test = y_poly[:-100]
# Process single crystal data separately
# X_mono = images_mono_train_test.reshape(images_mono_train_test[0], -1)
X_train_mono, X_test_mono, y_train_mono, y_test_mono = train_test_split(images_mono_train_test, y_mono_train_test, test_size=0.75, random_state=42,stratify=y_mono_train_test)

# # Train the SVM classifier
svm_classifier_mono = svm.SVC(C=0.5 ,kernel='linear',gamma=0.008,decision_function_shape='ovo')
svm_classifier_mono.fit(X_train_mono, y_train_mono)

# Process polycrystalline data separately
# X_poly = images_poly_train_test.reshape(images_poly_train_test.shape[0], -1)
X_train_poly, X_test_poly, y_train_poly, y_test_poly = train_test_split(images_poly_train_test, y_poly_train_test, test_size=0.75, random_state=42,stratify=y_poly_train_test)

# Train the SVM classifier
svm_classifier_poly = svm.SVC(C=0.5 ,kernel='linear',gamma=0.008,decision_function_shape='ovo')
svm_classifier_poly.fit(X_train_poly, y_train_poly)

# Predict and evaluate single crystal data
y_pred_mono = svm_classifier_mono.predict(X_test_mono)
print("mono data classification Report:")
print(confusion_matrix(y_test_mono, y_pred_mono))
print(classification_report(y_test_mono, y_pred_mono))

y_mono_valp = svm_classifier.predict(images_mono_val)
print("Verify the mono performance of the set:")
print(confusion_matrix(y_mono_val, y_mono_valp))
print(classification_report(y_mono_val, y_mono_valp))

# Prediction and evaluation of polycrystalline data
y_pred_poly = svm_classifier_poly.predict(X_test_poly)
print("poly data classification Report:")
print(confusion_matrix(y_test_poly, y_pred_poly))
print(classification_report(y_test_poly, y_pred_poly))

y_poly_valp = svm_classifier.predict(images_poly_val)
print("Verify the poly performance of the set:")
print(confusion_matrix(y_poly_val, y_poly_valp))
print(classification_report(y_poly_val, y_poly_valp))

mono data classification Report:
[[344  20   7  16]
 [ 42  30   3   2]
 [ 24   0   6   7]
 [100  19  17  94]]
              precision    recall  f1-score   support

           0       0.67      0.89      0.77       387
           1       0.43      0.39      0.41        77
           2       0.18      0.16      0.17        37
           3       0.79      0.41      0.54       230

    accuracy                           0.65       731
   macro avg       0.52      0.46      0.47       731
weighted avg       0.66      0.65      0.63       731

Verify the mono performance of the set:
[[66  5  1  0]
 [10  4  0  1]
 [ 6  0  0  0]
 [ 5  0  0  2]]
              precision    recall  f1-score   support

           0       0.76      0.92      0.83        72
           1       0.44      0.27      0.33        15
           2       0.00      0.00      0.00         6
           3       0.67      0.29      0.40         7

    accuracy                           0.72       100
   macro avg       0.47     

In [3]:
from sklearn import preprocessing
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from PIL import Image
import numpy as np
import os
from elpv_reader import load_dataset

   
def map_probability_to_label(prob):
    if prob == 0:
        return 0
    elif 0 < prob <= 0.4:
        return 1
    elif 0.4 < prob <= 0.8:
        return 2
    else:
        return 3


# Assuming you have a function to load your dataset
images, probs, types= load_dataset()
# print(images)
# print(image_height)

y = np.array([map_probability_to_label(prob) for prob in probs])
X = images.reshape(images.shape[0], -1)
# Split dataset into training and testing sets
X_val = X[-200:]
y_val = y[-200:]
X_train_test = X[:-200]
y_train_test = y[:-200]
X_train, X_test, y_train, y_test = train_test_split(X_train_test, y_train_test, test_size=0.75, random_state=42,stratify=y_train_test)
svm_classifier = svm.SVC(C=0.5 ,kernel='linear',gamma=0.008,decision_function_shape='ovo')
svm_classifier.fit(X_train, y_train)


y_pred = svm_classifier.predict(X_test)

# Calculate confusion matrix and classification report
print("data classification Report:")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
y_pred_val = svm_classifier.predict(X_val)
print("Verify the performance of the set:")
print(confusion_matrix(y_val, y_pred_val))
print(classification_report(y_val, y_pred_val))

data classification Report:
[[787  64  31 141]
 [104  65   5  24]
 [ 40  11   5  21]
 [199  58   7 256]]
              precision    recall  f1-score   support

           0       0.70      0.77      0.73      1023
           1       0.33      0.33      0.33       198
           2       0.10      0.06      0.08        77
           3       0.58      0.49      0.53       520

    accuracy                           0.61      1818
   macro avg       0.43      0.41      0.42      1818
weighted avg       0.60      0.61      0.60      1818

Verify the performance of the set:
[[139   1   0   4]
 [ 29   2   0   0]
 [  4   0   0   0]
 [ 18   0   3   0]]
              precision    recall  f1-score   support

           0       0.73      0.97      0.83       144
           1       0.67      0.06      0.12        31
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00        21

    accuracy                           0.70       200
   macro avg       0.3

In [4]:
from skimage.feature import hog
import matplotlib.pyplot as plt
import cv2

# 读取图像
def feature_extraction(image):
    if len(image.shape) == 3 and image.shape[2] == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image  # If it is already a grayscale image, no conversion is required
    

# Calculate the HOG feature of the image
    fd, hog_image = hog(gray_image, orientations=4, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), visualize=True, feature_vector=True)
    return fd
# plt.imshow(hog_image, cmap='gray')
# fd is the feature descriptor and hog_image is the visualized HOG image
# print(fd)

In [6]:
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from PIL import Image
import numpy as np
import os
from elpv_reader import load_dataset

   
def map_probability_to_label(prob):
    if prob == 0:
        return 0
    elif 0 < prob <= 0.4:
        return 1
    elif 0.4 < prob <= 0.8:
        return 2
    else:
        return 3


# Assuming you have a function to load your dataset
images, probs, types= load_dataset()
# print(images)
# print(image_height)

y = np.array([map_probability_to_label(prob) for prob in probs])
X = np.array([feature_extraction(image) for image in images])
# X = images.reshape(images.shape[0], -1)
# Split dataset into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75, random_state=42,stratify=y)
svm_classifier = svm.SVC(C=0.5 ,kernel='linear',gamma=0.008,decision_function_shape='ovo')
svm_classifier.fit(X_train, y_train)


y_pred = svm_classifier.predict(X_test)

# Calculate confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[1003   21    8   99]
 [ 172   27    0   22]
 [  55    4    1   20]
 [ 208   24    2  302]]
              precision    recall  f1-score   support

           0       0.70      0.89      0.78      1131
           1       0.36      0.12      0.18       221
           2       0.09      0.01      0.02        80
           3       0.68      0.56      0.62       536

    accuracy                           0.68      1968
   macro avg       0.46      0.40      0.40      1968
weighted avg       0.63      0.68      0.64      1968

